## IMPORTING THE REQUIRED LIBRARIES

In [1]:
import os
import numpy as np
from PIL import Image
import pandas as pd
from tensorflow.keras.models import load_model

## PUTTING THE PIXELS OF EVERY PHOTO IN A DATAFRAME

In [2]:
def importImages(path):

    images = os.listdir(path)

    data = list()

    for image in images:

        row = list()

        start = image.find('') + len('')
        end = image.find('.jpg')
        name = image[start:end]
        row.append(str(name))

        imageObject = Image.open(f'{path}/{image}')
        imageResized = imageObject.resize((48, 48))
        imageArray = np.asarray(imageResized)

        pixelsRows = list()

        for pixelsRow in imageArray:
            for pixel in pixelsRow:
                pixelsRows.append(pixel[0])
    
        row.append(pixelsRows)

        data.append(row)
        
    return pd.DataFrame(data = data, columns = ['name', 'pixels'])

images = importImages('facesImages')

## PREPARING THE PIXELS COLUMN FOR THE AGE NEURAL NETWORK

In [3]:
def formatPixelsAge(pixels: str) -> np.array:
    return np.asarray(pixels, dtype = np.uint8).reshape(48, 48)

pixelsAge = images['pixels'].apply(formatPixelsAge)
pixelsAge = np.reshape(pixelsAge.to_list(), (len(pixelsAge), 48, 48, 1)) / 255.0

## IMPORTING THE CLASSIFIER AND PREDICTING THE AGE OF THE FACES

In [4]:
ageClassifier = load_model('../models/age.h5')
agePredictions = ageClassifier.predict(pixelsAge)

## PREPARING THE PIXELS COLUMN FOR THE GENDER NEURAL NETWORK

In [5]:
def formatPixelsGender(pixels: str) -> np.array:

    pixelsArrayReshaped = np.asarray(pixels, dtype = np.uint8).reshape(48, 48)
    pixelsNormalized = pixelsArrayReshaped / 255.0

    return pixelsNormalized

pixelsGender = images['pixels'].apply(formatPixelsGender)
pixelsGender = np.reshape(pixelsGender.to_list(), (len(pixelsGender), 48, 48, 1)) / 255.0

## IMPORTING THE CLASSIFIER AND PREDICTING THE GENDER OF THE FACES

In [6]:
genderClassifier = load_model('../models/gender.h5')
genderPredictions = genderClassifier.predict(pixelsGender)

## INSERTING ALL THE PREDICTIONS IN THE IMAGES DATAFRAME

In [13]:
images['gender'] = [genderPrediction[0] for genderPrediction in genderPredictions]
images['age'] = [agePrediction[0] for agePrediction in agePredictions]

## PRESENTING THE PREDICTIONS

In [15]:
images

,name,pixels,gender,age
0,gonzalo,"[183, 185, 186, 187, 188, 189, 189, 190, 190, ...",0.457389,28.918571
1,ines,"[209, 208, 206, 203, 197, 160, 150, 191, 193, ...",0.372151,19.638681
2,jan,"[211, 226, 239, 239, 218, 184, 125, 169, 226, ...",0.955243,41.281910
3,jorien,"[151, 97, 96, 102, 104, 104, 106, 109, 108, 13...",0.173031,27.831879
